In [1]:
#Import common package

# General
import pandas as pd
import numpy as np
import re
import string
import random
import matplotlib.pyplot as plt

# PyTorch
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [32]:
# Import dataset puisi.csv

from google.colab import drive
drive.mount('/content/drive')


df = pd.read_csv("/content/drive/MyDrive/project/data/puisi.csv")

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,puisi,title,author,puisi_with_header
0,Tiada lagi resah di hatiku \n Tiada lagi kerag...,UNTUK KEKASIHKU,Oleh Suzianty Westerveld,UNTUK KEKASIHKU \n Oleh Suzianty Westerveld \n...
1,"Gemericik senandung luluh merayu hati, \n Terj...",HUJAN,Oleh Muslihin Abdillah,HUJAN \n Oleh Muslihin Abdillah \n \n Gemeric...
2,Kau adalah separuh hidup ku \n Hanya kepadamu ...,KAU,Oleh Nur Rahma Utami,KAU \n Oleh Nur Rahma Utami \n Kau adalah sepa...
3,Mungkin kau membiarkan aku \n Membiarkan peras...,TEPERANGKAP,Oleh Mangku Langit Jingga,TEPERANGKAP \n Oleh Mangku Langit Jingga \n \...
4,Ibu.. \n Kau adalah mutiara hatiku \n Mutiara ...,IBU,Oleh Dedeh Kurnia Afrianti,IBU \n Oleh Dedeh Kurnia Afrianti \n \n Ibu.....
...,...,...,...,...
7218,Rasa kangen ini \n Tak pernah terharap sedikit...,BUKAN PILIHAN HATIMU,Oleh Arshya Kyran,BUKAN PILIHAN HATIMU \n Oleh Arshya Kyran \n ...
7219,"Dititik manakah aku berada,. \n Dalam jiwamu h...",TANYA,Oleh Wahyu Dwi Septiaji,TANYA \n Oleh Wahyu Dwi Septiaji \n \n Dititi...
7220,Semoga apa yang kau inginkanakan terjadi dimas...,AYAH,Oleh Kendra,AYAH \n Oleh Kendra \n Semoga apa yang kau ing...
7221,Sahabat..... \n Maafkan aku Setelah Di sekolah...,SAHABAT YANG TERLUKA,Oleh Mutia,SAHABAT YANG TERLUKA \n Oleh Mutia \n \n Saha...


In [ ]:
!pip install jcopdl

In [ ]:
import jcopdl

In [33]:
df = df.drop(columns=["author", "puisi_with_header"])

In [34]:
# Hapus baris kosong atau NaN
df = df.dropna()

# Lihat jumlah data
print(f"total puisi {len(df)}")

total puisi 6901


In [39]:
df


,title,puisi,combined
0,UNTUK KEKASIHKU,Tiada lagi resah di hatiku \n Tiada lagi kerag...,UNTUK KEKASIHKU: Tiada lagi resah di hatiku \n...
1,HUJAN,"Gemericik senandung luluh merayu hati, \n Terj...","HUJAN: Gemericik senandung luluh merayu hati, ..."
2,KAU,Kau adalah separuh hidup ku \n Hanya kepadamu ...,KAU: Kau adalah separuh hidup ku \n Hanya kepa...
3,TEPERANGKAP,Mungkin kau membiarkan aku \n Membiarkan peras...,TEPERANGKAP: Mungkin kau membiarkan aku \n Mem...
4,IBU,Ibu.. \n Kau adalah mutiara hatiku \n Mutiara ...,IBU: Ibu.. \n Kau adalah mutiara hatiku \n Mut...
...,...,...,...
7218,BUKAN PILIHAN HATIMU,Rasa kangen ini \n Tak pernah terharap sedikit...,BUKAN PILIHAN HATIMU: Rasa kangen ini \n Tak p...
7219,TANYA,"Dititik manakah aku berada,. \n Dalam jiwamu h...","TANYA: Dititik manakah aku berada,. \n Dalam j..."
7220,AYAH,Semoga apa yang kau inginkanakan terjadi dimas...,AYAH: Semoga apa yang kau inginkanakan terjadi...
7221,SAHABAT YANG TERLUKA,Sahabat..... \n Maafkan aku Setelah Di sekolah...,SAHABAT YANG TERLUKA: Sahabat..... \n Maafkan ...


In [37]:
df = df[['title', 'puisi']]

In [38]:
#df = df[['title', 'puisi']].dropna()
#df = df.head(2000)
df['combined'] = df['title'] + ": " + df['puisi']
texts = df['combined'].tolist()

In [10]:
texts

['UNTUK KEKASIHKU: Tiada lagi resah di hatiku \n Tiada lagi keraguan kini \n Aku yakin hanyalah dirimu \n Yang ku sayangi \n Setiap langkah akupunBerjanji..................... \n Akan selalau setia bersama \n Kaulah nafasku \n Kaulah khayalanku \n Kaulah damai dalam detak jantungkuOh.............. \n kasih \n Peluklah diriku \n Bisikanlah apa yang kau mauOh.............. \n kasih \n Aku tak sanggup lagi \n Hidup tanpa dirimu \n  \n ',
 'HUJAN: Gemericik senandung luluh merayu hati, \n Terjamah larutan mensiasati. \n  \n Terus menyabani hingga rengkarnasi. \n Putih sejuk teramat berarti. \n Senantiasa menanti keputusan Tuhan kapan ia akan pergi. \n  \n Bila sampai waktu nya, \n Tetes demi tetes kian mengurai \xa0awan. \n Gemuruh kilat pelepas ikatan. \n Melepas hasrat bercinta tak terhelakan, \n Rayuan pulang tak hiraukan. \n Merintih seduh sedan tiada guna. \n  \n Hingga hujan mengiklaskan semua \n Demi senyuman yang menghianatinya, \n Yang tak terima akan kehadirannya \n Tanpa menyada

Tokenisasi

In [16]:
# ====================== LIBRARY ======================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import pandas as pd
from tqdm import tqdm

# ====================== DEVICE ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ====================== LOAD & PREPARE DATA ======================
# Asumsi df sudah dibaca sebelumnya: df = pd.read_csv(...)

# Ambil hanya 2000 puisi pertama
#df = df[['title', 'puisi']].dropna()
#df = df.head(2000)
#df['combined'] = df['title'] + ": " + df['puisi']
#texts = df['combined'].tolist()

# Tokenisasi (dan batasi panjang puisi ke 30 kata)
MAX_PUISI_LEN = 30
tokens = [sentence.lower().split()[:MAX_PUISI_LEN] for sentence in texts]

# Hitung vocab (top 5000 kata)
all_words = [word for sentence in tokens for word in sentence]
word_counts = Counter(all_words)
most_common = word_counts.most_common(7000)
vocab = [w for w, _ in most_common]

# Buat kamus kata → index
word2idx = {w: i+1 for i, w in enumerate(vocab)}  # mulai dari 1
word2idx['<pad>'] = 0
word2idx['<unk>'] = len(word2idx)
idx2word = {i: w for w, i in word2idx.items()}
vocab_size = len(word2idx)
print("Total vocab size:", vocab_size)

# Buat sequences (input, target)
MAX_INPUT_LEN = 30
sequences = []
for sentence in tqdm(tokens, desc="Membuat sequences"):
    encoded = [word2idx.get(w, word2idx['<unk>']) for w in sentence]
    for i in range(1, len(encoded)):
        input_seq = encoded[:i]
        if len(input_seq) > MAX_INPUT_LEN:
            continue
        target = encoded[i]
        sequences.append((input_seq, target))

# Batasin jumlah data agar training ringan (optional)
sequences = sequences[:10000]
print("Total training samples:", len(sequences))

# Convert ke tensor
inputs = [torch.tensor(seq[0], dtype=torch.long) for seq in sequences]
targets = [torch.tensor(seq[1], dtype=torch.long) for seq in sequences]
padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
targets = torch.stack(targets)

# DataLoader
batch_size = 64
dataset = TensorDataset(padded_inputs, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# ====================== MODEL ======================
class PoemLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=256, num_layers=2, dropout=0.2):
        super(PoemLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, x):
        embed = self.embedding(x)
        lstm_out, _ = self.lstm(embed)
        last_output = lstm_out[:, -1, :]
        out = self.fc(last_output)
        return out

# ====================== TRAINING ======================
model = PoemLSTM(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200
for epoch in range(1, epochs + 1):
    print(f"\n▶️ Epoch {epoch} mulai...")
    model.train()
    total_loss = 0

    for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"✅ Epoch {epoch} selesai - Loss: {avg_loss:.4f}")

    if epoch % 50 == 0:
        torch.save(model.state_dict(), f"puisi_lstm_epoch{epoch}.pth")
        print(f"💾 Model saved: puisi_lstm_epoch{epoch}.pth")

# ====================== SAVE MODEL ======================
torch.save(model.state_dict(), "puisi_lstm_final.pth")
print("🎉 Model disimpan sebagai 'puisi_lstm_final.pth'")


Using device: cuda
Total vocab size: 5002


Membuat sequences: 100%|██████████| 2000/2000 [00:00<00:00, 36331.94it/s]

Total training samples: 10000



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(



▶️ Epoch 1 mulai...
✅ Epoch 1 selesai - Loss: 7.2223

▶️ Epoch 2 mulai...
✅ Epoch 2 selesai - Loss: 6.6274

▶️ Epoch 3 mulai...
✅ Epoch 3 selesai - Loss: 6.5207

▶️ Epoch 4 mulai...
✅ Epoch 4 selesai - Loss: 6.4290

▶️ Epoch 5 mulai...
✅ Epoch 5 selesai - Loss: 6.3025

▶️ Epoch 6 mulai...
✅ Epoch 6 selesai - Loss: 6.1416

▶️ Epoch 7 mulai...
✅ Epoch 7 selesai - Loss: 5.9459

▶️ Epoch 8 mulai...
✅ Epoch 8 selesai - Loss: 5.7193

▶️ Epoch 9 mulai...
✅ Epoch 9 selesai - Loss: 5.4810

▶️ Epoch 10 mulai...
✅ Epoch 10 selesai - Loss: 5.2105
🎉 Model disimpan sebagai 'puisi_lstm_final.pth'


In [42]:
# ====================== LIBRARY ======================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import pandas as pd
from tqdm import tqdm

# ====================== DEVICE ======================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ====================== LOAD & PREPARE DATA ======================
# Asumsi df sudah dibaca sebelumnya: df = pd.read_csv(...)

# Ambil hanya 2000 puisi pertama
#df = df[['title', 'puisi']].dropna()
#df = df.head(2000)
#df['combined'] = df['title'] + ": " + df['puisi']
#texts = df['combined'].tolist()

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from tqdm import tqdm

# -----------------------------
# 💡 Hyperparameters
# -----------------------------
MAX_PUISI_LEN = 30       # Maksimal panjang puisi (per baris)
MAX_INPUT_LEN = 30       # Maksimal panjang input sequence ke LSTM
VOCAB_LIMIT = 7000       # Top N kata paling umum
BATCH_SIZE = 64          # Ukuran batch
SEQUENCE_LIMIT = None    # Ganti ke 10000 untuk debug cepat (atau None untuk full)

# -----------------------------
# 🧹 Tokenisasi & Truncation
# -----------------------------
tokens = [sentence.lower().split()[:MAX_PUISI_LEN] for sentence in texts]

# -----------------------------
# 🧮 Hitung Vocab (Top N Kata)
# -----------------------------
all_words = [word for sentence in tokens for word in sentence]
word_counts = Counter(all_words)
most_common = word_counts.most_common(VOCAB_LIMIT)
vocab = [w for w, _ in most_common]

# -----------------------------
# 🔤 Mapping Kata ↔ Index
# -----------------------------
word2idx = {w: i + 1 for i, w in enumerate(vocab)}  # Mulai dari index 1
word2idx['<pad>'] = 0
word2idx['<unk>'] = len(word2idx)

idx2word = {i: w for w, i in word2idx.items()}
vocab_size = len(word2idx)
print("✅ Total vocab size:", vocab_size)

# -----------------------------
# 📐 Buat Sequence Input → Target
# -----------------------------
sequences = []
for sentence in tqdm(tokens, desc="🧩 Membuat sequences"):
    encoded = [word2idx.get(w, word2idx['<unk>']) for w in sentence]
    for i in range(1, len(encoded)):
        input_seq = encoded[:i]
        if len(input_seq) > MAX_INPUT_LEN:
            continue
        target = encoded[i]
        sequences.append((input_seq, target))

# 🔻 Batasi jumlah data (optional)
if SEQUENCE_LIMIT:
    sequences = sequences[:SEQUENCE_LIMIT]
print("✅ Total training samples:", len(sequences))

# -----------------------------
# 📦 Siapkan TensorDataset
# -----------------------------
inputs = [torch.tensor(seq[0], dtype=torch.long) for seq in sequences]
targets = [torch.tensor(seq[1], dtype=torch.long) for seq in sequences]

padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
targets = torch.stack(targets)

dataset = TensorDataset(padded_inputs, targets)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


# ====================== MODEL ======================
class PoemLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=256, num_layers=2, dropout=0.2):
        super(PoemLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, x):
        embed = self.embedding(x)
        lstm_out, _ = self.lstm(embed)
        last_output = lstm_out[:, -1, :]
        out = self.fc(last_output)
        return out

# ====================== TRAINING ======================
model = PoemLSTM(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200
for epoch in range(1, epochs + 1):
    print(f"\n▶️ Epoch {epoch} mulai...")
    model.train()
    total_loss = 0

    for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"✅ Epoch {epoch} selesai - Loss: {avg_loss:.4f}")

    if epoch % 50 == 0:
        torch.save(model.state_dict(), f"puisi_lstm_epoch{epoch}.pth")
        print(f"💾 Model saved: puisi_lstm_epoch{epoch}.pth")

# ====================== SAVE MODEL ======================
torch.save(model.state_dict(), "puisi_lstm_final.pth")
print("🎉 Model disimpan sebagai 'puisi_lstm_final.pth'")


Using device: cuda
✅ Total vocab size: 7002


🧩 Membuat sequences: 100%|██████████| 6901/6901 [00:01<00:00, 5857.12it/s]


✅ Total training samples: 199993

▶️ Epoch 1 mulai...
✅ Epoch 1 selesai - Loss: 6.6302

▶️ Epoch 2 mulai...
✅ Epoch 2 selesai - Loss: 6.4518

▶️ Epoch 3 mulai...
✅ Epoch 3 selesai - Loss: 6.1840

▶️ Epoch 4 mulai...
✅ Epoch 4 selesai - Loss: 5.8768

▶️ Epoch 5 mulai...
✅ Epoch 5 selesai - Loss: 5.5631

▶️ Epoch 6 mulai...
✅ Epoch 6 selesai - Loss: 5.2448

▶️ Epoch 7 mulai...
✅ Epoch 7 selesai - Loss: 4.9259

▶️ Epoch 8 mulai...
✅ Epoch 8 selesai - Loss: 4.6123

▶️ Epoch 9 mulai...
✅ Epoch 9 selesai - Loss: 4.3087

▶️ Epoch 10 mulai...
✅ Epoch 10 selesai - Loss: 4.0321

▶️ Epoch 11 mulai...
✅ Epoch 11 selesai - Loss: 3.7738

▶️ Epoch 12 mulai...
✅ Epoch 12 selesai - Loss: 3.5406

▶️ Epoch 13 mulai...
✅ Epoch 13 selesai - Loss: 3.3275

▶️ Epoch 14 mulai...
✅ Epoch 14 selesai - Loss: 3.1315

▶️ Epoch 15 mulai...
✅ Epoch 15 selesai - Loss: 2.9545

▶️ Epoch 16 mulai...
✅ Epoch 16 selesai - Loss: 2.8022

▶️ Epoch 17 mulai...
✅ Epoch 17 selesai - Loss: 2.6622

▶️ Epoch 18 mulai...
✅ Epoch 18 

KeyboardInterrupt: 

In [45]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from tqdm import tqdm

# -----------------------------
# 💡 Hyperparameters
# -----------------------------
MAX_PUISI_LEN = 30       # Maksimal panjang puisi (per baris)
MAX_INPUT_LEN = 30       # Maksimal panjang input sequence ke LSTM
VOCAB_LIMIT = 7000       # Top N kata paling umum
BATCH_SIZE = 64          # Ukuran batch
SEQUENCE_LIMIT = None    # Ganti ke 10000 untuk debug cepat (atau None untuk full)

# -----------------------------
# 🧹 Tokenisasi & Truncation
# -----------------------------
tokens = [sentence.lower().split()[:MAX_PUISI_LEN] for sentence in texts]

# -----------------------------
# 🧮 Hitung Vocab (Top N Kata)
# -----------------------------
all_words = [word for sentence in tokens for word in sentence]
word_counts = Counter(all_words)
most_common = word_counts.most_common(VOCAB_LIMIT)
vocab = [w for w, _ in most_common]

# -----------------------------
# 🔤 Mapping Kata ↔ Index
# -----------------------------
word2idx = {w: i + 1 for i, w in enumerate(vocab)}  # Mulai dari index 1
word2idx['<pad>'] = 0
word2idx['<unk>'] = len(word2idx)

idx2word = {i: w for w, i in word2idx.items()}

import pickle

with open("word2idx.pkl", "wb") as f:
    pickle.dump(word2idx, f)
with open("idx2word.pkl", "wb") as f:
    pickle.dump(idx2word, f)


In [46]:
model = PoemLSTM(len(word2idx)).to(device)
model.load_state_dict(torch.load("puisi_lstm_epoch100.pth"))
model.eval()

PoemLSTM(
  (embedding): Embedding(7002, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=7002, bias=True)
)

In [17]:
import torch.nn.functional as F

def generate_poem(seed_text, model, word2idx, idx2word, max_len=20, temperature=1.0):
    model.eval()
    words = seed_text.lower().split()
    input_seq = [word2idx.get(w, word2idx['<unk>']) for w in words]

    for _ in range(max_len):
        input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            output = output.squeeze()

            # Sampling dengan temperature (opsional)
            output = output / temperature
            probs = F.softmax(output, dim=-1).cpu()
            predicted_id = torch.multinomial(probs, num_samples=1).item()

        predicted_word = idx2word.get(predicted_id, "<unk>")
        if predicted_word == "<pad>":
            break
        words.append(predicted_word)
        input_seq.append(predicted_id)

        # Batasi panjang input untuk performa
        if len(input_seq) > 30:
            input_seq = input_seq[-30:]

    return ' '.join(words)


In [47]:
import torch.nn.functional as F

def generate_poem(seed_text, model, word2idx, idx2word, max_len=20, temperature=1.0, top_k=50):
    model.eval()
    words = seed_text.lower().split()
    input_seq = [word2idx.get(w, word2idx['<unk>']) for w in words]

    unk_id = word2idx.get('<unk>')
    pad_id = word2idx.get('<pad>')

    for _ in range(max_len):
        input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            output = output.squeeze() / temperature
            probs = F.softmax(output, dim=-1).cpu()

            # Top-k sampling
            top_probs, top_idx = torch.topk(probs, top_k)
            top_probs = top_probs / top_probs.sum()  # re-normalisasi
            predicted_id = top_idx[torch.multinomial(top_probs, 1).item()].item()

        # Hindari pad dan unk di output
        if predicted_id in [pad_id, unk_id]:
            continue

        predicted_word = idx2word.get(predicted_id, "<unk>")
        words.append(predicted_word)
        input_seq.append(predicted_id)

        # Batasi panjang input agar tetap efisien
        if len(input_seq) > 30:
            input_seq = input_seq[-30:]

    return ' '.join(words)


In [52]:
#seed = "untuk wanita yang ku cintai"
#generated = generate_poem(seed, model, word2idx, idx2word, max_len=20)
#print("\n📝 Generated Puisi:\n", generated)

generate_poem("tuhan dan aku", model, word2idx, idx2word)


'tuhan dan aku tetap cintaku padamu sejuknya karena ketulusan maupun pesan engkau adalah penerang aku sumpah dan ingin selalu'

In [55]:
import torch.nn.functional as F

def generate_poem(seed_text, model, word2idx, idx2word, max_len=20, temperature=1.0):
    model.eval()
    words = seed_text.lower().split()
    input_seq = [word2idx.get(w, word2idx['<unk>']) for w in words]

    for _ in range(max_len):
        input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_tensor)
            output = output.squeeze()
            output = output / temperature
            probs = F.softmax(output, dim=-1).cpu()

            # Hindari <unk> dengan set probabilitasnya ke 0
            unk_idx = word2idx.get('<unk>')
            if unk_idx is not None:
                probs[unk_idx] = 0
                probs = probs / probs.sum()  # re-normalisasi

            predicted_id = torch.multinomial(probs, num_samples=1).item()


        predicted_word = idx2word.get(predicted_id, "<unk>")
        if predicted_word == "<pad>":
            break
        words.append(predicted_word)
        input_seq.append(predicted_id)
        if len(input_seq) > 30:
            input_seq = input_seq[-30:]

    return ' '.join(words)


def generate_4_lines_poem(seed_text, model, word2idx, idx2word, max_len=20, temperature=0.9):
    raw_poem = generate_poem(seed_text, model, word2idx, idx2word, max_len=max_len, temperature=temperature)
    words = raw_poem.strip().split()
    per_line = max(1, len(words) // 4)
    formatted = []

    for i in range(0, len(words), per_line):
        line = ' '.join(words[i:i+per_line]).capitalize()
        if i + per_line >= len(words):
            line += '.'  # Akhiran puisi
        else:
            line += ','  # Akhiran larik
        formatted.append(line)

    return '\n'.join(formatted)


In [59]:
print(generate_4_lines_poem("tentang rindu", model, word2idx, idx2word, max_len=24))

Tentang rindu dan menangis kan menyapa,
Kasih orang yang menyayangi dan bunga,
Semua bahkan ini sadarlah rasa rinduku,
Tuk merasa bahagia dan membawa bahagia,
Yang senantiasa.


In [ ]:
from collections import Counter

# Tokenisasi manual (split berdasarkan spasi)
tokens = [sentence.lower().split() for sentence in texts]
all_words = [word for sentence in tokens for word in sentence]

# Buat kamus kata → index
word_counts = Counter(all_words)
vocab = sorted(word_counts.keys())
word2idx = {w: i+1 for i, w in enumerate(vocab)}  # index dimulai dari 1
word2idx['<pad>'] = 0

# Untuk decode balik
idx2word = {i: w for w, i in word2idx.items()}

vocab_size = len(word2idx)
print("Total vocab size:", vocab_size)


Total vocab size: 64535


Encoded input -> output sequence

In [ ]:
sequences = []

for sentence in tokens:
    encoded = [word2idx[w] for w in sentence if w in word2idx]
    for i in range(1, len(encoded)):
        input_seq = encoded[:i]  # urutan sebelumnya
        target = encoded[i]      # kata berikutnya
        sequences.append((input_seq, target))


Padding sequence

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch

# Ubah ke tensor
inputs = [torch.tensor(seq[0], dtype=torch.long) for seq in sequences]
targets = [torch.tensor(seq[1], dtype=torch.long) for seq in sequences]

# Pad input sequences agar panjang sama
padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
targets = torch.stack(targets)

In [ ]:
# Dataset and Dataloader
batch_size = 64

dataset = TensorDataset(padded_inputs, targets)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


Model Arsitektur

In [ ]:
class PoemLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size=128, hidden_size=256, num_layers=2, dropout=0.2):
        super(PoemLSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, vocab_size)  # x2 karena bidirectional

    def forward(self, x):
        embed = self.embedding(x)                # (batch, seq_len, embed_size)
        lstm_out, _ = self.lstm(embed)           # (batch, seq_len, hidden*2)
        last_output = lstm_out[:, -1, :]         # Ambil output terakhir
        out = self.fc(last_output)               # (batch, vocab_size)
        return out

In [ ]:
# Trainin preparation or mcoc
model = PoemLSTM(vocab_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.2/343.2 kB 11.4 MB/s eta 0:00:00


In [ ]:
from ignite.engine import Engine, Events
from ignite.handlers import EarlyStopping, ModelCheckpoint


In [ ]:
def train_step(engine, batch):
    model.train()
    x, y = batch
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()

    return loss.item()

trainer = Engine(train_step)


In [ ]:
# Checkpoint handler
checkpoint = ModelCheckpoint(
    dirname="model",
    filename_prefix="best_puisi_lstm",
    n_saved=1,
    create_dir=True,
    require_empty=False,
    score_function=lambda engine: -engine.state.output,
    score_name="val_loss"
)

# Early stopping handler
early_stopping = EarlyStopping(
    patience=3,
    score_function=lambda engine: -engine.state.output,  # negasi loss
    trainer=trainer
)

# Register ke trainer
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint, {'model': model})
trainer.add_event_handler(Events.EPOCH_COMPLETED, early_stopping)


In [ ]:
loss_history = []

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training(engine):
    epoch = engine.state.epoch
    loss = engine.state.output
    loss_history.append(loss)
    print(f"> Epoch {epoch} - Loss: {loss:.4f}")


In [ ]:
# Jalankan training
trainer.run(dataloader, max_epochs=200)

# Simpan model manual setelah training selesai
torch.save(model.state_dict(), "model/puisi_lstm_final.pth")

# Simpan dictionary jika belum disimpan
import pickle
with open("model/word2idx.pkl", "wb") as f:
    pickle.dump(word2idx, f)
with open("model/idx2word.pkl", "wb") as f:
    pickle.dump(idx2word, f)

# Setelah training, tampilkan grafik loss

#plt.figure(figsize=(8, 5))
#plt.plot(loss_history, label="Training Loss")
#plt.xlabel("Epoch")
#plt.ylabel("Loss")
#plt.title("Training Loss Over Epochs")
#plt.legend()
#plt.grid(True)
#plt.show()


ERROR:ignite.engine.engine.Engine:Engine run is terminating due to exception: 


KeyboardInterrupt: 

In [ ]:
def generate_poem(model, seed_text, word2idx, idx2word, max_len=20):
    model.eval()
    words = seed_text.lower().split()
    input_seq = [word2idx.get(w, 0) for w in words]  # pakai 0 kalau kata tidak dikenal

    for _ in range(max_len):
        input_tensor = torch.tensor([input_seq], dtype=torch.long).to(device)

        with torch.no_grad():
            output = model(input_tensor)  # [batch_size=1, vocab_size]
            probs = torch.softmax(output[0], dim=0).cpu().numpy()

        next_word_id = np.random.choice(len(probs), p=probs)
        next_word = idx2word.get(next_word_id, "<unk>")
        input_seq.append(next_word_id)
        words.append(next_word)

        if next_word in [".", "!", "?", "\n"]:
            break

    return " ".join(words)

In [ ]:
poem = generate_poem(model, seed_text="wanita yang ku cintai", word2idx=word2idx, idx2word=idx2word)

print("Puisi hasil generate:")
print(poem)

Puisi hasil generate:
wanita yang ku cintai aku kau tak yang rupa yang minta tanpa tak maret cinta aku ku kang ku tak kantong diriku 31 seberkas
